## Importing Required Libraries


In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import os
from pathlib import Path

## Adding Indicators to Stock files

In [14]:
class Indicators:
    @staticmethod
    def add_rsi(df, period: int = 14, price_col: str = "close", out_col: str = "rsi"):
        prices = df[price_col].to_numpy(dtype=float)
        n = len(prices)
        rsi = np.full(n, np.nan, dtype=float)
        if n > period:
            deltas = np.diff(prices)
            gains = np.clip(deltas, a_min=0.0, a_max=None)
            losses = np.clip(-deltas, a_min=0.0, a_max=None)
            avg_gain = np.full(n, np.nan, dtype=float)
            avg_loss = np.full(n, np.nan, dtype=float)
            avg_gain[period] = gains[:period].mean()
            avg_loss[period] = losses[:period].mean()
            for i in range(period+1, n):
                gain = gains[i - 1]
                loss = losses[i - 1]
                avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain) / period
                avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss) / period
            rs = np.divide(avg_gain, avg_loss, out=np.full(n, np.nan, dtype=float), where=avg_loss != 0)
            rsi_vals = 100.0 - (100.0 / (1.0 + rs))
            rsi = rsi_vals
            rsi[:period] = np.nan
        df[out_col] = rsi
        return df

    def _ema(values: np.ndarray, period: int) -> np.ndarray:
        n = len(values)
        ema = np.full(n, np.nan, dtype=float)
        if n == 0 or period <= 0: return ema
        
        # --- FIND FIRST VALID INDEX ---
        first_valid_idx = np.where(~np.isnan(values))[0]
        if len(first_valid_idx) == 0:
            return ema # All NaNs
        
        start_idx = first_valid_idx[0]
        
        # The actual data we're smoothing
        valid_values = values[start_idx:] 
        m = len(valid_values)
        
        # Calculate EMA on the valid subset
        if m >= period:
            # Use a mean of the first 'period' valid values as the seed
            seed_idx_relative = period - 1
            seed = np.mean(valid_values[:period])
        else:
            # Use mean of all valid values if not enough for period
            seed_idx_relative = m - 1
            seed = np.mean(valid_values)
            
        # Set the seed in the full EMA array (absolute index)
        seed_idx_absolute = start_idx + seed_idx_relative
        ema[seed_idx_absolute] = seed
        
        alpha = 2.0 / (period + 1.0)
        prev = seed
        
        # Iterate from the point after the seed
        for i in range(seed_idx_absolute + 1, n):
            x = values[i]
            # Only update if the current value is not NaN
            if not np.isnan(x):
                prev = (x - prev) * alpha + prev
                ema[i] = prev
            # If x is NaN, the prev value carries forward, but the EMA at i is NaN (correct)
            
        return ema

    @staticmethod
    def add_macd(
        df, fast: int = 12, slow: int = 26, signal: int = 9,
        price_col: str = "close", macd_col: str = "macd",
        signal_col: str = "macd_signal", hist_col: str = "macd_hist",
    ):
        prices = df[price_col].to_numpy(dtype=float)
        ema_fast = Indicators._ema(prices, fast)
        ema_slow = Indicators._ema(prices, slow)
        macd = ema_fast - ema_slow
        signal_line = Indicators._ema(macd, signal)
        hist = macd - signal_line
        df[macd_col] = macd
        df[signal_col] = signal_line
        df[hist_col] = hist
        return df
    @staticmethod
    def _wilder_smooth(values: np.ndarray, period: int) -> np.ndarray:
        """
        Calculates Wilder's Smoothing (a specific type of EMA).
        alpha = 1 / period
        """
        n = len(values)
        smoothed = np.full(n, np.nan, dtype=float)
        
        # Find the first non-NaN value to start the calculation
        first_valid_idx_arr = np.where(~np.isnan(values))[0]
        if len(first_valid_idx_arr) == 0:
            return smoothed # All values are NaN

        first_valid_idx = first_valid_idx_arr[0]
        
        # The first smoothed value is the simple average of the first 'period' valid points
        start_calc_idx = first_valid_idx + period
        if start_calc_idx > n:
            return smoothed # Not enough data to calculate

        # Seed the first smoothed value
        smoothed[start_calc_idx - 1] = np.mean(values[first_valid_idx:start_calc_idx])

        # Apply Wilder's smoothing for subsequent values
        for i in range(start_calc_idx, n):
            if not np.isnan(values[i]):
                prev_smooth = smoothed[i - 1]
                smoothed[i] = (prev_smooth * (period - 1) + values[i]) / period
            # If current value is NaN, smoothed value remains NaN
        
        return smoothed
    @staticmethod
    def add_adx(
        df, period: int = 14, high_col: str = "high", low_col: str = "low", 
        close_col: str = "close", adx_col: str = "adx", pdi_col: str = "pdi", 
        ndi_col: str = "ndi"
    ):
        """
        Calculates and adds the Average Directional Index (ADX),
        Positive Directional Indicator (+DI), and Negative Directional Indicator (-DI).
        """
        high = df[high_col]
        low = df[low_col]
        close = df[close_col]
        n = len(df)

        # 1. Calculate True Range (TR) and Directional Movement (+DM, -DM)
        up_move = high.diff()
        down_move = -low.diff()
        
        pdm = np.where((up_move > down_move) & (up_move > 0), up_move, 0)
        ndm = np.where((down_move > up_move) & (down_move > 0), down_move, 0)

        tr_1 = pd.DataFrame(high - low)
        tr_2 = pd.DataFrame(abs(high - close.shift(1)))
        tr_3 = pd.DataFrame(abs(low - close.shift(1)))
        tr_df = pd.concat([tr_1, tr_2, tr_3], axis=1)
        tr = tr_df.max(axis=1).to_numpy()

        # 2. Smooth TR, +DM, and -DM using Wilder's Smoothing
        smoothed_tr = Indicators._wilder_smooth(tr, period)
        smoothed_pdm = Indicators._wilder_smooth(pdm, period)
        smoothed_ndm = Indicators._wilder_smooth(ndm, period)

        # 3. Calculate Directional Indicators (+DI, -DI)
        # Use np.divide for safe division (handles division by zero)
        pdi = 100 * np.divide(smoothed_pdm, smoothed_tr, out=np.full(n, np.nan), where=smoothed_tr!=0)
        ndi = 100 * np.divide(smoothed_ndm, smoothed_tr, out=np.full(n, np.nan), where=smoothed_tr!=0)

        # 4. Calculate the Directional Index (DX)
        di_sum = pdi + ndi
        di_diff = np.abs(pdi - ndi)
        dx = 100 * np.divide(di_diff, di_sum, out=np.full(n, np.nan), where=di_sum!=0)

        # 5. Calculate the Average Directional Index (ADX) by smoothing DX
        adx = Indicators._wilder_smooth(dx, period)

        # 6. Add results to the DataFrame
        df[pdi_col] = pdi
        df[ndi_col] = ndi
        df[adx_col] = adx
        
        return df

## The Intraday Backtesting Class


In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class IntradayRankingRunner:
    """
    Runs an intraday long/short backtest based on momentum ranking over a lookback period.
    For each 15-minute candle, it ranks stocks based on the prior 1-hour performance,
    goes long on the top performer and short on the bottom performer for the duration of that candle.
    """
    def __init__(self, momentum_dir, stock_data_dir, initial_cash, commission, risk_free_rate=0.0):
        self.momentum_dir = Path(momentum_dir)
        self.stock_data_dir = Path(stock_data_dir)
        self.initial_cash = initial_cash
        self.commission = commission
        self.risk_free_rate = risk_free_rate
        self.equity_curve = [self.initial_cash]
        self.trade_log = []
        self.timestamps_log = [pd.Timestamp.now()]
    

    def _get_tickers_from_file(self, file_path):
        """Loads the top 5 tickers from a momentum ranking CSV file."""
        df = pd.read_csv(file_path)
        return df.iloc[0:5, 0].tolist() if len(df) >= 5 else []

    def _load_and_align_data(self, tickers, date_str):
        """
        Loads 15-minute data and adds RSI , ADX indicator.
        """
        all_data = {}
        common_index = None
        
        target_date = pd.to_datetime(date_str).date()

        for ticker in tickers:
            try:
                file_path = self.stock_data_dir / f"{ticker.replace('.', '_')}.csv"
                df = pd.read_csv(file_path, parse_dates=['date'])
                
                if 'close' in df.columns:
                    df = Indicators.add_rsi(df, period=14)
                    df = Indicators.add_adx(df, period=14) 
                
                df.columns = df.columns.str.lower()
                df_day = df[df['date'].dt.date == target_date].set_index('date').sort_index()
                
                if not df_day.empty:
                    all_data[ticker] = df_day
                    if common_index is None:
                        common_index = df_day.index
                    else:
                        common_index = common_index.intersection(df_day.index)
            except Exception as e:
                print(f"[Warning] Could not process {ticker}: {e}")
                continue
        
        aligned_data = {ticker: df.reindex(common_index) for ticker, df in all_data.items()}
        return aligned_data, common_index

    def run_backtest(self):
        """Main method to run the backtest with dynamic, proportional allocation."""
        ranking_files = sorted(list(self.momentum_dir.glob("momentum_ranking_*.csv")))
        
        for i, ranking_file in enumerate(ranking_files):
            date_str = ranking_file.name.split('_')[-1].replace('.csv', '')
            print(f"--- ({i+1}/{len(ranking_files)}) Processing universe for date: {date_str} ---")

            tickers = self._get_tickers_from_file(ranking_file)
            if len(tickers) < 5:
                print("Not enough tickers in ranking file. Skipping day.")
                continue

            aligned_data, timestamps = self._load_and_align_data(tickers, date_str)
            
            if timestamps is None or timestamps.empty or len(aligned_data) < 2:
                print("Not enough aligned data to run backtest for this day. Skipping.")
                continue

            for candle_idx in range(4, len(timestamps)-2,2):
                    current_time = timestamps[candle_idx]
                    lookback_time = timestamps[candle_idx - 4]
                    yday_time = timestamps[candle_idx-1]
                    next_time = timestamps[candle_idx+1]
                    
                    performance = []
                    for ticker, df in aligned_data.items():
                        try:
                            lookback_price = df.loc[lookback_time, 'open']
                            yday_price = df.loc[yday_time,'close']
                            if pd.notna(lookback_price) and pd.notna(yday_price) and lookback_price != 0:
                                pct_change = (yday_price - lookback_price) / lookback_price
                                performance.append({'ticker': ticker, 'pct_change': pct_change})
                        except KeyError:
                            continue 

                    if len(performance) < 2:
                        continue 

                    # --- NEW: DYNAMIC ALLOCATION LOGIC ---
                    current_portfolio_value = self.equity_curve[-1]
                    net_pnl_for_candle = 0.0
                    longs_executed = 0
                    shorts_executed = 0
                    
                    long_candidates = [p for p in performance if p['pct_change'] > 0]
                    short_candidates = [p for p in performance if p['pct_change'] < 0]

                    total_abs_momentum = sum(p['pct_change'] for p in long_candidates) + \
                                        sum(abs(p['pct_change']) for p in short_candidates)
                    
                    if total_abs_momentum == 0:
                        self.equity_curve.append(current_portfolio_value)
                        continue

                    # A. Process all LONG candidates
                    for candidate in long_candidates:
                        try:
                            ticker = candidate['ticker']
                            df = aligned_data[ticker]
                            rsi = df.loc[yday_time, 'rsi']
                            adx = df.loc[yday_time, 'adx'] 
                            
                            if pd.notna(rsi) and rsi > 60:
                                weight = candidate['pct_change'] / total_abs_momentum
                                capital = current_portfolio_value * weight
                                entry_price = df.loc[current_time, 'open']
                                stopll = entry_price*(1.0 - 99.5/100)
                                second_exit = df.loc[current_time,'close']
                                if(stopll>second_exit) :
                                    exit_price = second_exit
                                else:
                                    exit_price = max(df.loc[next_time, 'close'],stopll)

                                pnl = (capital / entry_price) * (exit_price - entry_price)
                                fees = self.commission * (capital + (capital / entry_price * exit_price))
                                net_pnl_for_candle += (pnl - fees)
                                longs_executed += 1
                        except (KeyError, ZeroDivisionError):
                            continue

                    # B. Process all SHORT candidates
                    for candidate in short_candidates:
                        try:
                            ticker = candidate['ticker']
                            df = aligned_data[ticker]
                            rsi = df.loc[yday_time, 'rsi']
                            adx = df.loc[yday_time, 'adx'] 

                            if pd.notna(rsi) and rsi < 40:
                                weight = abs(candidate['pct_change']) / total_abs_momentum
                                capital = current_portfolio_value * weight
                                entry_price = df.loc[current_time, 'open']
                                stopls = entry_price*(1.0 + 0.5/100)
                                
                                second_exit = df.loc[current_time,'close']
                                if(stopls<second_exit):
                                    exit_price = second_exit
                                else:
                                    exit_price = min(df.loc[next_time, 'close'],stopls)
                                
                                pnl = (capital / entry_price) * (entry_price - exit_price)
                                fees = self.commission * (capital + (capital / entry_price * exit_price))
                                net_pnl_for_candle += (pnl - fees)
                                shorts_executed += 1
                        except (KeyError, ZeroDivisionError):
                            continue
                    
                    # C. Update Portfolio and Log
                    new_portfolio_value = current_portfolio_value + net_pnl_for_candle
                    self.equity_curve.append(new_portfolio_value)
                    self.timestamps_log.append(current_time)
                    
                    if net_pnl_for_candle != 0.0:
                        self.trade_log.append({
                            'datetime': current_time,
                            'num_longs': longs_executed,
                            'num_shorts': shorts_executed,
                            'net_pnl': net_pnl_for_candle,
                            'portfolio_value': new_portfolio_value
                        })


            print("\\n--- Intraday Backtest Finished ---")

    def _load_nifty_data(self, start_date, end_date):
        """Downloads and cleans Nifty 100 daily data from yfinance."""
        try:
            nifty_df = yf.download(
                tickers='^CNX100',
                start=start_date.strftime('%Y-%m-%d'),
                end=(end_date + pd.Timedelta(days=1)).strftime('%Y-%m-%d'),
                progress=False,
                auto_adjust = False
            )
            
            if nifty_df.empty:
                print("[Warning] No data for Nifty 100 in this period. Skipping benchmark.")
                return None

            if isinstance(nifty_df.columns, pd.MultiIndex):
                nifty_df.columns = nifty_df.columns.droplevel(1)
            
            return nifty_df[['Close']].rename(columns={'Close': 'close'})
            
        except Exception as e:
            print(f"[Warning] Could not download Nifty 100 data: {e}")
            return None

        
    def display_final_results(self):
        """Calculates and displays the final performance metrics and plots."""
        print("\n--- FINAL PORTFOLIO METRICS ---")

        if not self.trade_log:
            print("No trades were executed."); return

        # Get the actual first timestamp from the trade log
        first_trade_dt = self.trade_log[0]['datetime']

        equity_series = None
        # Ensure the lists are the same size, which they should be now
        if len(self.equity_curve) == len(self.timestamps_log):
            # Replace the initial placeholder timestamp with the real start time
            self.timestamps_log[0] = first_trade_dt
            # Create the series with a proper DatetimeIndex
            equity_series = pd.Series(self.equity_curve, index=pd.to_datetime(self.timestamps_log))
        else:
            # This is a fallback in case something goes wrong
            print(f"[Warning] Mismatch in logs. Equity: {len(self.equity_curve)}, Timestamps: {len(self.timestamps_log)}. Plotting will be affected.")
            equity_series = pd.Series(self.equity_curve)
        
        # Sort the index to ensure it's chronological, which is required for reindexing
        equity_series = equity_series.sort_index()

        if equity_series.empty or len(equity_series) <= 1:
            print("Not enough data to generate metrics.")
            return

        initial_balance = equity_series.iloc[0]
        final_balance = equity_series.iloc[-1]
        net_return = (final_balance / initial_balance) - 1
        
        if not self.trade_log:
            print("No trades were executed."); return

        first_trade_dt = self.trade_log[0]['datetime']
        last_trade_dt = self.trade_log[-1]['datetime']
        num_years = (last_trade_dt - first_trade_dt).total_seconds() / (365.25 * 24 * 60 * 60)
        annualized_return = (1 + net_return) ** (1 / num_years) - 1 if num_years > 0 else 0
        
        running_max = equity_series.cummax()
        drawdown = (running_max - equity_series) / running_max
        max_drawdown = drawdown.max()
        
        returns = equity_series.pct_change().dropna()
        excess_returns = returns - (self.risk_free_rate / (252 * 25))
        
        sharpe_ratio = 0
        if excess_returns.std() != 0:
            sharpe_ratio = (excess_returns.mean() / excess_returns.std()) * np.sqrt(252 * 25)

        sortino_ratio = 0
        downside_returns = excess_returns[excess_returns < 0]
        if not downside_returns.empty and downside_returns.std() != 0:
            sortino_ratio = (excess_returns.mean() / downside_returns.std()) * np.sqrt(252 * 25)

        # --- Nifty Calculation Block ---
        nifty_net_return = 0
        nifty_annualized_return = 0
        nifty_equity_curve = None
        nifty_data = self._load_nifty_data(first_trade_dt, last_trade_dt)

        if nifty_data is not None and not nifty_data.empty:
            nifty_start_price = nifty_data['close'].iloc[0]
            nifty_end_price = nifty_data['close'].iloc[-1]
            
            nifty_net_return = ((nifty_end_price / nifty_start_price) - 1).item()
            
            nifty_annualized_return = (1 + nifty_net_return) ** (1 / num_years) - 1 if num_years > 0 else 0
            
            initial_investment = 100000
            nifty_returns = nifty_data['close'].pct_change().fillna(0)
            nifty_growth_factor = (1 + nifty_returns).cumprod()
            nifty_equity_curve = initial_investment * nifty_growth_factor

        # --- Print statements ---
        print(f"Initial Balance:          {initial_balance:,.2f}")
        print(f"Final Balance:            {final_balance:,.2f}")
        print(f"Total Return (ROI):       {net_return:.2%}")
        print(f"Annualized Return:        {annualized_return:.2%}")
        print(f"Nifty 100 Total Return:   {nifty_net_return:.2%}")
        print(f"Nifty 100 Annualized:     {nifty_annualized_return:.2%}")
        print(f"Max Drawdown:             {max_drawdown:.2%}")
        print(f"Sharpe Ratio:             {sharpe_ratio:.2f}")
        print(f"Sortino Ratio:            {sortino_ratio:.2f}")
        print(f"Total Trades:             {len(self.trade_log)}")
        
        # --- Plotting logic ---
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, 
                            subplot_titles=("Equity Curve Comparison", "Drawdown"))
    
        # 1. Plot your strategy's equity curve
        fig.add_trace(go.Scatter(x=equity_series.index, y=equity_series.values, mode='lines', name="Strategy Equity"), row=1, col=1)

        # 2. Add the Nifty 100 equity curve plot
        if nifty_equity_curve is not None:
            # This forward-fills the daily Nifty value for each 15-minute interval.
            nifty_equity_curve_aligned = nifty_equity_curve.reindex(equity_series.index, method='ffill')
            
            fig.add_trace(go.Scatter(x=nifty_equity_curve_aligned.index, y=nifty_equity_curve_aligned.values, mode='lines', name="Nifty 100 (₹100k)", line_color='orange'), row=1, col=1)

        # 3. Plot the drawdown
        fig.add_trace(go.Scatter(x=equity_series.index, y=drawdown * 100, mode='lines', name="Drawdown", fill='tozeroy', line_color='red'), row=2, col=1)
        
        # Update layout with new titles
        fig.update_layout(title="Intraday Ranking Strategy Performance", height=800, template="plotly_dark")
        fig.update_yaxes(title_text="Portfolio Value (₹)", row=1, col=1)
        fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=1) 
        fig.show()


    def display_trade_log(self):
            """Displays the detailed log of all intraday trades, including win/loss stats."""
            print("\n--- INTRADAY TRADE LOG ---")
            if not self.trade_log:
                print("No trades were executed during the backtest.")
                return

            log_df = pd.DataFrame(self.trade_log)

            # Print the detailed log table
            print("\n--- Full Trade History (per 15min candle) ---")
            print(log_df.to_string())


## Main Execution

In [ ]:
# --- CONFIGURATION & EXECUTION --- 
# Define directories and parameters 

MOMENTUM_FOLDER = Path("backtest_data_15min") 
STOCK_DATA_FOLDER = Path("combined_stock_data") # <-- MUST CONTAIN 15-MIN DATA
INITIAL_CASH = 100000 
COMMISSION_RATE = 0.000 # Set a realistic commission for high-frequency trading
ANNUAL_RISK_FREE_RATE = 0.065

# Initialize and run the new intraday backtester 
intraday_runner = IntradayRankingRunner(
    momentum_dir=MOMENTUM_FOLDER,
    stock_data_dir=STOCK_DATA_FOLDER,
    initial_cash=INITIAL_CASH,
    commission=COMMISSION_RATE,
    risk_free_rate=ANNUAL_RISK_FREE_RATE
)

# Run the backtest
intraday_runner.run_backtest()

# Display the final results and plots
intraday_runner.display_final_results()

# Display the detailed trade log
intraday_runner.display_trade_log()

--- (1/264) Processing universe for date: 2024-09-02 ---
\n--- Intraday Backtest Finished ---
--- (2/264) Processing universe for date: 2024-09-03 ---
\n--- Intraday Backtest Finished ---
--- (3/264) Processing universe for date: 2024-09-04 ---
\n--- Intraday Backtest Finished ---
--- (4/264) Processing universe for date: 2024-09-05 ---
\n--- Intraday Backtest Finished ---
--- (5/264) Processing universe for date: 2024-09-06 ---
\n--- Intraday Backtest Finished ---
--- (6/264) Processing universe for date: 2024-09-09 ---
\n--- Intraday Backtest Finished ---
--- (7/264) Processing universe for date: 2024-09-10 ---
\n--- Intraday Backtest Finished ---
--- (8/264) Processing universe for date: 2024-09-11 ---
\n--- Intraday Backtest Finished ---
--- (9/264) Processing universe for date: 2024-09-12 ---
\n--- Intraday Backtest Finished ---
--- (10/264) Processing universe for date: 2024-09-13 ---
\n--- Intraday Backtest Finished ---
--- (11/264) Processing universe for date: 2024-09-16 ---
\


--- INTRADAY TRADE LOG ---

--- Full Trade History (per 15min candle) ---
                datetime  num_longs  num_shorts      net_pnl  portfolio_value
0    2024-09-02 10:15:00          2           1  -357.077545     99642.922455
1    2024-09-02 10:45:00          1           0   -30.255413     99612.667042
2    2024-09-02 11:15:00          1           0   -82.865184     99529.801858
3    2024-09-02 11:45:00          1           0    13.007415     99542.809273
4    2024-09-02 12:15:00          1           0     8.247968     99551.057241
5    2024-09-02 12:45:00          2           1  -122.158540     99428.898700
6    2024-09-02 13:15:00          3           0   855.349516    100284.248216
7    2024-09-02 13:45:00          2           0  2085.635297    102369.883513
8    2024-09-02 14:15:00          3           0   896.408515    103266.292029
9    2024-09-02 14:45:00          3           0 -1053.830589    102212.461440
10   2024-09-03 10:15:00          1           0    59.877825    102